In [12]:
import cvxpy as cp
import numpy as np

# 汇率矩阵 (5x5): a[i][k] = 1 unit of currency i -> a[i][k] units of currency k
a = np.array([
    [1.0,      0.8946,   0.751968, 1.300303, 103.834605],
    [1.117714, 1.0,      0.84304,  1.453597, 116.079975],
    [1.325714, 1.186,    1.0,      1.726699, 138.0915],
    [0.768620, 0.687324, 0.579102, 1.0,      79.855125],
    [0.009628, 0.0086,   0.007241, 0.012516, 1.0]
])

n = 5  # 5 种货币

# 决策变量
x = cp.Variable((n, n), name="x")  # x[i][j]: 从 i 到 j 的转换金额
y = cp.Variable(n, name="y")       # y[k]: 货币 k 的净余额

# 目标函数：最大化 USD 的净余额
objective = cp.Maximize(y[0])  # y1 对应 USD

# 约束条件
constraints = []

# 流守恒约束：对于每个货币 k
for k in range(n):
    inflow = sum(a[i][k] * x[i][k] for i in range(n))  # 所有流入 k 的金额
    outflow = sum(x[k][j] for j in range(n))           # 所有从 k 流出的金额
    constraints.append(y[k] == inflow - outflow)

# y1 <= 1：总净收益不能超过 1（防止无界）
constraints.append(y[0] <= 1)

# 非负性
constraints.extend([x[i][j] >= 0 for i in range(n) for j in range(n)])
constraints.extend([y[k] >= 0 for k in range(n)])

In [13]:
# 构建并求解问题
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS, verbose=False)

# 输出结果
print("状态:", problem.status)
print("最大净 USD 收益:", problem.value)

if problem.value > 1e-6:
    print("\n✅ 存在套利机会！")
else:
    print("\n❌ 不存在套利机会。")

# 显示交易路径
print("\nx_value\n")
print(x.value)
print("\ny_value\n")
print(y.value)

状态: optimal
最大净 USD 收益: 0.9999999500194565

✅ 存在套利机会！

x_value

[[1069.86883717 3757.27156056  158.83059316  321.30277574   19.00286544]
 [ 556.89739547 1069.86883717 4167.4471061   280.24269692   18.1634904 ]
 [ 408.90219805  549.62134395 1069.86883717 3025.33596161   18.34769384]
 [3993.77484083 1442.60691582  613.92923393 1069.86883717   25.33298243]
 [2406.85424042 2122.69003132 1946.1497513  2151.08690708 1069.86883717]]

y_value

[ 0.99999995  0.14885318  0.17265193  0.27451703 11.42065078]
